# Svensson (1994)



$$ r(t) = \beta_1 + \beta_2\frac{1-e^{-\lambda_1 t}}{\lambda_1 t} 
        + \beta_3 \left(\frac{1-e^{-\lambda_1 t}}{\lambda_1 t}-e^{-\lambda_1 t}\right)
        + \beta_4 \left(\frac{1-e^{-\lambda_2 t}}{\lambda_2 t}-e^{-\lambda_2 t}\right)$$

### Importing the data

In [1]:
from scipy.optimize import fmin
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import datetime as dt
from bizdays import Calendar

path = r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\Brasil\ETTJ\Base_LTN.xlsx"
path2 = r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\Brasil\ETTJ\Parametros.xlsx"
df = pd.read_excel(path)
parameters = pd.read_excel(path2)

In [2]:
cal = Calendar.load("ANBIMA")
parameters.set_index("Data referência", inplace=True)

### Maturity Calculation

df["Maturity"] = np.array(cal.bizdays(df["DATA_REFERENCIA"], df["DATA_VENCIMENTO"]))/252
df.set_index("DATA_REFERENCIA", inplace=True)

### Duration Calculation

cpn = 0
yld = df["EXPECTATIVA"]
m = 1
n = df["Maturity"]

df["Macaulay_duration"] = ((1+yld) / (m*yld)) - ( (1 + yld + n*(cpn-yld)) / ((m*cpn* ((1+yld)**n - 1)) + m*yld) )

In [3]:
df.head()

,CODIGO,DATA_VENCIMENTO,EXPECTATIVA,PU,Maturity,Macaulay_duration
DATA_REFERENCIA,,,,,,
2023-07-03,LTN,2023-10-01,13.2025,968.996662,0.250000,0.250000
2023-07-03,LTN,2024-01-01,12.7468,942.225059,0.492063,0.492063
2023-07-03,LTN,2024-04-01,12.1642,918.761537,0.738095,0.738095
2023-07-03,LTN,2024-07-01,11.5298,897.786879,0.988095,0.988095
2023-07-03,LTN,2024-10-01,11.0232,877.471104,1.250000,1.250000


### Generating individuals

In [4]:
N =1200             ### Number of individuals 
p = int(2/3*N)      ### Proportion of individuals generated by method 1
var = 0.5           ### variance normal distribution

### 1 - Initial Values of the results of the estimation of the previous day

ref_date_str = "2023-07-11"
ref_date = datetime.strptime(ref_date_str, "%Y-%m-%d")
previous_date = cal.offset(ref_date_str,-1)
previous_parameters = parameters.loc[str(previous_date)][1:]
beta_star_1 = np.array([previous_parameters.replace(previous_parameters[1],previous_parameters[0])])
e_previous = beta_star_1.T*np.random.normal(0, var, size=(6, p))
beta_previous = beta_star_1.T + e_previous

### 2 - Approximation that takes into account the observed yield to maturity (ytm)

data_estimada = df.loc[ref_date_str]
data_estimada.sort_values("Maturity")
b1 = data_estimada["EXPECTATIVA"][0]/100
b2 = data_estimada["EXPECTATIVA"][-1]/100 - data_estimada["EXPECTATIVA"][0]/100
b3 = 0
b4 = 0
lbda1 = (data_estimada["Maturity"][-1]-data_estimada["Maturity"][0])/2
lbda2 = lbda1
beta_star_2 = np.array([b1, b2, b3 , b4, lbda1, lbda2])[:, np.newaxis]
e_approx = beta_star_2*np.random.normal(0, var, size=(6, N-p))
beta_approx = beta_star_2 + e_approx ## ajustar colunas

### Adding constraints to the parameters

def update (betas,beta_star):
    updated_value = []
    for index, parameter in enumerate(betas):
        if parameter[0]<0:
            betas[index][0] = beta_star[0]+ beta_star[0]*np.random.normal(0, var)
            updated_value.append(betas[index][0]<0)
        if (parameter[0]+parameter[1])<0:
            betas[index][1] = beta_star[1]+ beta_star[1]*np.random.normal(0, var)
            updated_value.append((betas[index][0]+betas[index][1])<0)
        if parameter[4]<0:
            betas[index][4] = beta_star[4]+ beta_star[4]*np.random.normal(0, var)
            updated_value.append(betas[index][4]<0)
        if parameter[5]<0:
            betas[index][5] = beta_star[5]+ beta_star[5]*np.random.normal(0, var)
            updated_value.append(betas[index][5]<0)
    if sum(updated_value) != 0:
        update(betas,beta_star)
    return(beta_previous.T)

update(beta_previous.T,beta_star_1.T)
update(beta_approx,beta_star_2)

individuals = np.concatenate((beta_previous.T, beta_approx.T), axis=0)

### Selection

In [5]:
### add duration and coupons
alpha = 6
var = 0.5
pi = 0.35
### 40% of the individuals survive
s = int(0.4 * N)


for interation in range(5):

    values = []
    
    for row in individuals:
        def myval(c):
            nss =(c[0])+(c[1]*((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))+(c[2]*((((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))-(np.exp(-df['Maturity']*c[4]))))+(c[3]*((((1-np.exp(-df['Maturity']*c[5]))/(df['Maturity']*c[5])))-(np.exp(-df['Maturity']*c[5]))))
            Calculated_price  = 1000 / (1 + nss) ** df['Maturity'] 
            df['Residual'] =  (df['PU'] - Calculated_price)**2*(1/df["Macaulay_duration"]**0.5)
            val = np.sum(df['Residual'])
            return(val)
    
        val = myval(row)
        values.append((val, row)) 
    sol = pd.DataFrame(values, columns=['SC', "Parameters"]).sort_values('SC')

    selection = sol.sort_values('SC')[0:s]


    next_gen = []
    e_mutation = []
    
    if interation > 1:
        var = var*1.02

    for num in range(N):
        psi = np.random.uniform(0, 1)
        theta_r = np.concatenate(selection.iloc[(np.random.beta(1,alpha, 1)*s),:]["Parameters"].values)
        theta_s = np.concatenate(selection.iloc[(np.random.beta(1,alpha, 1)*s),:]["Parameters"].values)
        next_gen.append(psi*theta_r+(1-psi)*theta_s)
        e = np.random.normal(0, var, 6)*np.random.choice([0, 1], size=6, p=[1 - pi, pi])
        e_mutation.append(e)

    individuals = np.array(next_gen) + np.array(next_gen)*np.array(e_mutation)

selection.head()

,SC,Parameters
119,348.752412,"[0.11562896391807664, 0.02874698799468447, 0.0..."
493,418.609071,"[0.11092493024107283, 0.027524411701347246, 0...."
171,424.464017,"[0.11680754022751536, 0.033205777645214, 0.040..."
925,445.059827,"[0.10822755825748963, 0.04376164432895183, 0.0..."
723,448.002733,"[0.1086349048494158, 0.03250187039259035, 0.05..."


In [ ]:

dic[f"{ref_date_str}"]=selection.iloc[0]

In [ ]:
curves = pd.DataFrame(dic)
curves

In [ ]:
rounded_series = selection["Parameters"].apply(lambda lst: [round(x, 1) for x in lst])
rounded_series[0:99]

In [ ]:
rounded_series.apply(lambda x: x == rounded_series.iloc[0]).all()

In [ ]:
np.round(selection["SC"][0:99], decimals = 0)